In [2]:
import pandas as pd
from sampling import *
from sklearn.preprocessing import StandardScaler
import preprocess_data
from autogluon.tabular import TabularDataset, TabularPredictor

2025-03-24 11:23:13.234115: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742811793.294589  272654 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742811793.313274  272654 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742811793.440936  272654 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742811793.440959  272654 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742811793.440961  272654 computation_placer.cc:177] computation placer alr

In [3]:
# Load your dataset
df = pd.read_csv('task/train_set/x_train_aggregated.csv') # Replace with your file path
x = df.drop(columns=['AccountID'])
y_df = pd.read_csv('task/train_set/y_train.csv')
# df['Fraudster'] = y_df['Fraudster']
xy_train = pd.merge(df, y_df, on='AccountID', how='inner')

df_val = pd.read_csv('task/validation_set/x_val_aggregated.csv')
x_val = df_val.drop(columns=['AccountID'])
yval_df = pd.read_csv('task/validation_set/y_val.csv')
# df_val['Fraudster'] = yval_df['Fraudster']
xy_val = pd.merge(df_val, yval_df, on='AccountID', how='inner')
# y_val = yval_df['Fraudster']
# Display the first few rows
# print(df.head())

test_df = pd.read_csv('task/test_set/x_test.csv')
test_agg_df = pd.read_csv('task/test_set/x_test_aggregated.csv')

# Check the distribution of the target variable (Fraudster)
# print(df_val['Fraudster'].value_counts())

xtrain_df = pd.read_csv('task/train_set/x_train.csv')
x_val_df = pd.read_csv('task/validation_set/x_val.csv')

In [4]:
df_fea, df_val_fea, df_test_fea = preprocess_data.preprocess_and_extract_features()


df_train_fea shape: (11061, 25)
df_val_fea shape: (11060, 25)
df_test_fea shape: (11060, 24)


In [4]:
# print("Training Data:", df_fea.head())
# print("Validation Data:", df_val_fea.head())
# print("Test Data:", df_test_fea.head())

In [5]:
# from sklearn.feature_selection import mutual_info_classif
# import matplotlib.pyplot as plt

# # df_fe3 = pd.merge(df_fe, df, on='AccountID', how='inner')
# X = df_fea.drop(columns=['AccountID','Fraudster'])
# y = df_fea['Fraudster']

# x_val = df_val_fea.drop(columns=['AccountID','Fraudster'])
# y_val = df_val_fea['Fraudster']

# X_test = df_test_fea.drop(columns=['AccountID'])

# # Calculate Mutual Information scores
# mi_scores = mutual_info_classif(X, y)

# # Create a DataFrame to store feature names and their MI scores
# mi_df = pd.DataFrame({'Feature': X.columns, 'MI_Score': mi_scores})
# mi_df = mi_df.sort_values(by='MI_Score', ascending=False)

# # Plot the MI scores
# plt.figure(figsize=(10, 6))
# plt.barh(mi_df['Feature'], mi_df['MI_Score'])
# plt.xlabel('Mutual Information Score')
# plt.title('Feature Importance (Mutual Information)')
# plt.show()

# # Select top k features
# k = 10 # Number of top features to select
# top_features_mi = mi_df['Feature'][:k].tolist()
# print("Top Features (Mutual Information):", top_features_mi)

In [6]:
# # Combine selected features from all methods
# selected_features = list(set(top_features_mi ))
# # selected_features = list(set(top_features_mi).intersection(selected_features_l1))
# print("Final Selected Features:", selected_features)

In [7]:
# df_fea = df_fea[selected_features + ['AccountID', 'Fraudster']]
# df_val_fea = df_val_fea[selected_features+ ['AccountID', 'Fraudster']]
# df_test_fea = df_test_fea[selected_features+ ['AccountID']]

In [8]:
# df_fea.head()

In [9]:
# fraud_detector = FraudDetection(df_fea,df_val_fea)
# fraud_detector.preprocess_data()
# scaler = StandardScaler()

# # Generate oversampled data
# X_train_smote, y_train_smote = fraud_detector.oversample_data()
# X_scaled = scaler.fit_transform(x_val)
# x_val = pd.DataFrame(scaler.transform(x_val), columns=x_val.columns)

# X_train_smote


In [10]:
# X_train_smote_df = pd.DataFrame(X_train_smote, columns=selected_features)

In [11]:
# X_train_smote['Fraudster'] = y_train_smote

In [12]:
# df_fea.to_csv('task/autogluon/x_train.csv', index=False)
# df_val_fea.to_csv('task/autogluon/x_val.csv', index=False)
# df_test_fea.to_csv('task/autogluon/x_test.csv', index=False)

In [13]:
train_data = TabularDataset('task/autogluon/x_train.csv')
train_data.head()

label = 'Fraudster'
problem_type = 'binary'
eval_metric = 'f1'
#print(train_data[label].describe())
predictor = TabularPredictor(label=label, problem_type=problem_type, eval_metric=eval_metric ).fit(train_data, num_bag_sets=2, num_bag_folds=10, num_stack_levels=2)


No path specified. Models will be saved in: "AutogluonModels/ag-20250324_075725"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.0
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #53~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Wed Jan 15 19:18:46 UTC 2
CPU Count:          16
Memory Avail:       6.17 GB / 14.87 GB (41.5%)
Disk Space Avail:   11.68 GB / 176.11 GB (6.6%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in co

Beginning AutoGluon training ...
AutoGluon will save models to "/home/zoro/Desktop/BBDC/bbdc2025-SSH_ML/AutogluonModels/ag-20250324_075725"
Train Data Rows:    11061
Train Data Columns: 24
Label Column:       Fraudster
Problem Type:       binary
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    6313.16 MB
	Train Data (Original)  Memory Usage: 2.57 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 1 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryM

In [14]:
# predictor = TabularPredictor(label=label, problem_type=problem_type, eval_metric=eval_metric)

# # Fit the model with updated arguments
# predictor.fit(
#     train_data=train_data,
#     presets='best_quality',  # Use high-quality presets for better performance
#     auto_stack=True,  # Enable stacking for better performance
#     num_bag_folds=10,  # Increase bagging folds for robustness
#     num_bag_sets=5,    # Use multiple bagging sets
#     fit_weighted_ensemble=True,  # Enable weighted ensembles
#     calibrate_decision_threshold='auto',  # Automatically calibrate decision threshold for F1
#   # Set a time limit (e.g., 1 hour)
# )


In [15]:
# Calibrate the decision threshold for F1 score
predictor.calibrate_decision_threshold(metric='f1')

Calibrating decision threshold to optimize metric f1 | Checking 51 thresholds...
Calibrating decision threshold via fine-grained search | Checking 38 thresholds...
	Base Threshold: 0.500	| val: 0.6198
	Best Threshold: 0.390	| val: 0.6269


0.39

In [16]:
test_data = TabularDataset('task/autogluon/x_val.csv')

y_pred = predictor.predict(test_data.drop(columns=[label]))

predictor.evaluate(test_data, silent=True)

Loaded data from: task/autogluon/x_val.csv | Columns = 25 / 25 | Rows = 11060 -> 11060
/home/zoro/anaconda3/envs/bbdc/lib/python3.12/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")
/home/zoro/anaconda3/envs/bbdc/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/

{'f1': 0.5958155016642891,
 'accuracy': 0.8462929475587704,
 'balanced_accuracy': 0.810094005822668,
 'mcc': 0.5245705524752448,
 'roc_auc': 0.9184396670290393,
 'precision': 0.49060297572435396,
 'recall': 0.7584745762711864}

In [17]:
print(predictor.leaderboard(test_data))

/home/zoro/anaconda3/envs/bbdc/lib/python3.12/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")
/home/zoro/anaconda3/envs/bbdc/lib/python3.12/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary 

                      model  score_test  score_val eval_metric  \
0     NeuralNetTorch_BAG_L2    0.598002   0.593015          f1   
1     NeuralNetTorch_BAG_L3    0.596537   0.592092          f1   
2       WeightedEnsemble_L4    0.595816   0.619788          f1   
3       WeightedEnsemble_L3    0.595672   0.609365          f1   
4    NeuralNetFastAI_BAG_L2    0.593663   0.603797          f1   
5       WeightedEnsemble_L2    0.592468   0.575054          f1   
6     ExtraTreesGini_BAG_L2    0.592332   0.542802          f1   
7    NeuralNetFastAI_BAG_L3    0.591429   0.614515          f1   
8            XGBoost_BAG_L3    0.589334   0.565928          f1   
9     ExtraTreesGini_BAG_L3    0.588688   0.557000          f1   
10          CatBoost_BAG_L3    0.588200   0.556943          f1   
11    ExtraTreesEntr_BAG_L3    0.586879   0.561404          f1   
12        LightGBMXT_BAG_L3    0.586714   0.554559          f1   
13          CatBoost_BAG_L2    0.585938   0.555891          f1   
14  Random

In [18]:
y_pred = predictor.predict(test_data.drop(columns=[label]))
print(len(y_pred))  

/home/zoro/anaconda3/envs/bbdc/lib/python3.12/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")
/home/zoro/anaconda3/envs/bbdc/lib/python3.12/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary 

11060


In [19]:
xy_pred = pd.DataFrame(y_pred)
xy_pred['AccountID'] = df_val_fea['AccountID'].values
xy_pred_merge = pd.merge(yval_df, xy_pred, on='AccountID', how='inner')
xy_pred_merge.to_csv('task/autogluon/y_val_model_pred.csv', index=False)

y_true = xy_pred_merge['Fraudster_x']
y_predy = xy_pred_merge['Fraudster_y']

NameError: name 'df_val_fea' is not defined

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
print("Confusion Matrix:\n", confusion_matrix(y_true, y_predy))
print("\nClassification Report:\n", classification_report(y_true, y_predy))


Confusion Matrix:
 [[7977 1431]
 [ 372 1280]]

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.85      0.90      9408
           1       0.47      0.77      0.59      1652

    accuracy                           0.84     11060
   macro avg       0.71      0.81      0.74     11060
weighted avg       0.88      0.84      0.85     11060



In [ ]:
#predicting over x_test
# test_data_up = TabularDataset('task/autogluon/x_test.csv')
# y_test_pred = predictor.predict(test_data_up)
# print(len(y_test_pred))


In [ ]:
# xy_test_pred = pd.DataFrame(y_test_pred)
# xy_test_pred['AccountID'] = df_test_fea['AccountID'].values
# xskel = pd.read_csv('task/student_skeleton.csv')
# xskel = xskel[['AccountID']]
# xy_test_pred_merge = pd.merge(xskel, xy_test_pred, on='AccountID', how='inner')
# xy_test_pred_merge.to_csv('submission_files/Submission_5.csv', index=False)
xy_test_pred_merge.head()

NameError: name 'xy_test_pred_merge' is not defined

In [ ]:
xy_test_pred_merge['Fraudster'].value_counts()

Fraudster
0    8552
1    2508
Name: count, dtype: int64

In [ ]:
predictor.feature_importance(test_data)

These features in provided data are not utilized by the predictor and will be ignored: ['AccountID']
Computing feature importance via permutation shuffling for 23 features using 5000 rows with 5 shuffle sets...
/home/zoro/anaconda3/envs/bbdc/lib/python3.12/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")
	24.29s	= Expected runtime (4.86s per shuffle set)
/home/zoro/anaconda3/envs/bbdc/lib/python3.12/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute

,importance,stddev,p_value,n,p99_high,p99_low
TotalOut,0.242276,0.016900,0.000003,5,0.277075,0.207478
TotalIn,0.235500,0.017243,0.000003,5,0.271004,0.199996
ExternalAccountFrequency_mean,0.178436,0.015858,0.000007,5,0.211089,0.145784
TimeSinceLastTransaction_mean,0.104389,0.006952,0.000002,5,0.118703,0.090075
MaxDebit,0.067848,0.011545,0.000097,5,0.091619,0.044076
IsUnusualTransactionType_sum,0.050486,0.007746,0.000064,5,0.066435,0.034538
AvgAmount,0.043984,0.009902,0.000288,5,0.064372,0.023596
RollingTransactionCount_mean,0.029485,0.010242,0.001498,5,0.050574,0.008396
TransactionFrequencyPerHour_max,0.029429,0.008743,0.000834,5,0.047431,0.011427
MaxFreqH,0.023016,0.008729,0.002070,5,0.040988,0.005043


In [1]:
from tabpfn import TabPFNClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the dataset
X = df_fea.drop(columns=['Fraudster'])
y = df_fea['Fraudster']
X_val = df_val_fea.drop(columns=['Fraudster'])
y_val = df_val_fea['Fraudster']

classifier = TabPFNClassifier(
    ignore_pretraining_limits=True  # Bypass default dataset size limits
)

# Fit the model
classifier.fit(X, y)



NameError: name 'df_fea' is not defined

In [6]:
# Evaluate
y_pred = classifier.predict(X_val)
print('Test Accuracy:', accuracy_score(y_val, y_pred))

: 